<a href="https://colab.research.google.com/github/akashshingha850/Biosignal-Processing-1/blob/main/BioSignal_Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 3: Adaptive filtering

## Learning Outcomes
After this assignment, students can use adaptive filters to remove interfering signals summed with the desired signal. The concept of adaptive filtering is first approached through more simple linear optimization models, and students also learn basic vector mathematics, cross-correlation, and interpolation.

## Required Reading
Read chapters 3.6-3.9, especially chapter 3.9, from the course book.

## Assignment Description
Your task is to implement an adaptive filtering solution to obtain fetal ECG by canceling out the maternal ECG from the measurement.

## Data
The file 'signals.mat' contains all the signals for the assignment. The variables contained in the file are listed below:

- Fetus signal: 'fhb' (pure fetus signal used to assess analysis results)
- Mother’s chest signal: 'mhb' (pure mother’s signal used to assess analysis results)
- Abdomen signals 'abd_sig1', 'abd_sig2', and 'abd_sig3' (mixed signals from fetus and mother)
- A real respiration movement signal and R-to-R interval sequence from ECG signal: 'RespReference' and 'RRiInput'

The sampling frequency of `fhb`, `mhb`, `abd_si`


## 3.1 Pure sum of signals

### Background

Consider that we are observing a mixture of two signals

$$
x(t) = c_1 x_1(t) + c_2 x_2(t) = \sum_{i=1}^{2} c_i x_i(t). \, (1)
$$

where \( c_1 \neq 0 \) and \( c_2 \neq 0 \) are mixing coefficients. In addition, we can measure the second signal directly as

$$
y(t) = x_2(t)
$$

simultaneously.

Our task is to estimate \( x_1(t) \) from \( x(t) \) and \( y(t) \). Since it is impossible to make a distinction between different values for the coefficients \( c_1 \) and signal \( x_1(t) \) amplitudes, we can assume that without loss of generality that \( c_1 = 1 \). Thus, using the assumption that \( c_1 = 1 \) and \( y(t) = x_2(t) \), we can rewrite (1) as

$$
x(t) - c_2 y(t) = x_1(t). \, (2)
$$

In this case, these signals are ECGs coming from a pregnant mother and the fetus. In the measurements, we can obtain the mother's ECG from the chest separately as a reference, but the abdominal measurement aimed to measure fetus' ECG is contaminated with that of the mother's as depicted in the Figure below.

![ECG Measurement](https://lcms-files.mathworks.com/content/images/ecfcaded-a3cc-426f-9106-b7000b1a3a06.png)

For the numerical solution, we consider that we have \( N \) samples with sampling time interval \( T \) of the signals collected into corresponding vectors, i.e.

$$
\mathbf{x} = [x(0) \, x(T) \, x(2T) \, \cdots \, x((N-1)T)]^T,
$$
$$
\mathbf{x}_1 = [x_1(0) \, x_1(T) \, x_1(2T) \, \cdots \, x_1((N-1)T)]^T,
$$
$$
\mathbf{y} = [y(0) \, y(T) \, y(2T) \, \cdots \, y((N-1)T)]^T.
$$

Moreover, we are looking for optimal estimation in the l2-norm sense. Hence, we choose the value of \( c_2 \) that makes the vectors \( \mathbf{x} \) and \( \mathbf{y} \) most similar with each other, i.e. we minimize

$$
\min_{c_2} || \mathbf{x} - c_2 \mathbf{y} ||_2. \, (3)
$$

According to (2), then also \( || \mathbf{x}_1 ||_2 \) is minimized. Geometrically, due to the triangle inequality, this means that \( \mathbf{x}_1 \) and \( \mathbf{x}_2 \) are orthogonal, i.e.

$$
\mathbf{x}_1^T \mathbf{x}_2 = 0.
$$

See the Figure below.

![Orthogonality Illustration](https://lcms-files.mathworks.com/content/images/03bd1d36-e0e0-458d-895c-af1f09c9bfa6.png)

Please also note that orthogonality means that

$$
\sum_{i=0}^{N-1} x_1(iT) x_2(iT) = 0.
$$

Thus, \( \mathbf{x}_1 \) and \( \mathbf{x}_2 \) are also uncorrelated if either (or both) of their means are zero.

We [can compute](https://en.wikipedia.org/wiki/Vector_projection) \( c_2 \) via the scalar projection as

$$
c_2 = \frac{\mathbf{x}^T \mathbf{x}_2}{\mathbf{x}_2^T \mathbf{x}_2}, \, (4)
$$

and, \( \mathbf{x}_1 \) as the difference (vector rejection)

$$
\mathbf{x}_1 = \mathbf{x} - c_2 \mathbf{x}_2 = \mathbf{x} - \frac{\mathbf{x}^T \mathbf{x}_2}{\mathbf{x}_2^T \mathbf{x}_2} \mathbf{x}_2. \, (5)
$$

### Relation to Adaptive Filtering

Now, note that the solution (5) corresponds to the signal processing filter structure shown in the Figure below.

![Filter Structure](https://lcms-files.mathworks.com/content/images/7c5d86b6-35af-4d46-96e1-c14dc762b607.png)

Basically, we have here a one tap (adaptive) filter whose coefficient is matched by comparing the input signals in the l2-sense minimizing \( \mathbf{x}_1 \) which leads to minimizing the difference between \( \mathbf{x}_1 \) and \( c_2 \mathbf{x}_2 \).

Finally, please also note that (3) is the least squares formulation of the matrix equation

$$
\mathbf{x} = c_2 \mathbf{y} = c_2 \mathbf{x}_2, \, (6)
$$

where the vectors \( \mathbf{x} \) and \( \mathbf{y} = \mathbf{x}_2 \) are \( N \times 1 \) matrices. Hence, we can also obtain \( c_2 \) via [the Moore-Penrose pseudoinverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse#Linearly_independent_columns).

$$
c_2 = \mathbf{x}_2^+ \mathbf{x}.
$$

### Task

Load data from 'problem1.mat' file, and use the variables 'abd_sig1' and 'mhb'. The former vector contains an artificial mixture of maternal and fetal ECGs summed together in the abdomen measurement. The latter vector contains only the maternal ECG as measured from the chest.

Implement the scalar projection formula (4) to calculate the mixing coefficient \( c_2 \) directly. Then use the 'pinv' function in MATLAB as an alternative way to calculate the coefficient. Finally, also use the [backslash operator](https://se.mathworks.com/help/matlab/ref/mldivide.html) (\) to solve (6).


In [ ]:
"""
% Load problem1.mat to have access to variables abd_sig1 and mhb
load('problem1.mat');

% The sampling rates are 1000 Hz
FS = 1000;

% Calculate sample timing vector in seconds starting from 0
%%t =
t = (0:length(abd_sig1)-1) / FS;

% Estimate c2 from abd_sig1 and mhb using the scalar projection formula
%%c2_projection =
c2_projection = dot(abd_sig1', mhb) / dot(mhb', mhb);

% Estimate c2 from abd_sig1 and mhb using the pseudoinverse function (pinv)
c2_pinv = pinv(mhb) * abd_sig1;


% Estimate c2 from abd_sig1 and mhb using the backslash operator (\)
c2_operator = mhb \ abd_sig1;


% Calculate the fetal ECG by cancelling out the scaled mother's ECG using projection based estimation coefficient
%%fetus =
fetus = abd_sig1 - c2_projection * mhb;
"""

## 3.2 Delayed sum of signals
### Background

Consider now the same basic setting as in the previous problem.

This time, however, let us suppose that there is a small time delay between the signals so that the mother's signal in the abdomen -- that is \( x_2 \) in

$$
x(t) = x_1(t) + c_2 x_2(t)
$$

-- is delayed compared to the chest by \( d \) samples (integer!). We denote this as

$$
y(t) = x_2(t + \tau) = x_2(t + dT)
$$

where \( \tau = dT \) is the amount of time ahead the chest signal is.

Clearly, if we know the time difference, we time shift (delay) the chest signal, and then use the l2-norm based optimal rejection/cancellation method of the previous problem. For example, [cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation) can be used to find the time delay at which the signals correlate with each other the most.

### Relation to Adaptive Filtering

Now, note that the solution of the problem corresponds to the signal processing filter structure shown in the Figure below.

![Filter Structure](https://lcms-files.mathworks.com/content/images/9b84c2d7-e5d9-45a0-a799-237b046face6.png)

Basically, we have here a multitap (adaptive) filter whose all other coefficients are zero, but the one matching the delay is found optimally as in the first problem. The delay can be estimated using cross-correlation.

### Task

Load data from 'problem2.mat' file, and use the variables 'abd_sig1' and 'mhb_ahead'. The former vector contains an artificial mixture of maternal and fetal ECGs summed together in the abdomen measurement. The latter vector contains only the maternal ECG as measured from the chest.

First, fix the time delay by shifting the maternal ECG in the 'mhb_ahead' vector accordingly, storing the value in the vector 'mhb'. Pad the missing samples with the nearest existing value.

Then, as in the previous task, calculate the fetal ECG by cancelling out the mother's ECG.

### Note

Padding with the nearest value means that you need to use the first or last value of the signal to replace the empty samples that appear depending on the direction of the time shifting.


In [ ]:
"""
% Load problem2.mat to have access to variables abd_sig1 and mhb_ahead
load('problem2.mat');

% The sampling rates are 1000 Hz
FS = 1000;

% Calculate sample timing vector in seconds starting from 0
t = (0:length(abd_sig1)-1) / FS;

% Estimate the time lag using cross correlation
% (Calculate cross correlation using the xcorr function and then
% use the max function to find the lag giving maximal correlation)
[correlation, lag] = xcorr(abd_sig1, mhb_ahead);
[~, idx] = max(correlation);
d = lag(idx);

% Shift the chest ECG mhb_ahead back in time d samples padding with nearest value
%%mhb = circshift(mhb_ahead, -round(d), 1);
%%mhb = circshift(mhb_ahead, -d, 1);
mhb=ones(size(mhb_ahead));
mhb(d+1:end) = mhb_ahead(1:end-d);
mhb(1:d) = mhb(d+1);

% Estimate c2 from abd_sig1 and mhb
%%c2 = sum(abd_sig1 .* mhb) / sum(mhb.^2);
c2 = sum(abd_sig1 .* mhb) / sum(mhb .* mhb);

% Calculate the fetal ECG by cancelling out the scaled mother's ECG using projection based estimation coefficient
fetus = abd_sig1 - c2 * mhb;
"""

## 3.3 Delayed sum of signals - subsample accuracy

### Background

Consider now the same basic setting as in the previous problem. This time, however, let us suppose that there is a small time delay between the signals so that the mother's signal in the abdomen -- that is \(x_2\) in

$$
x(t) = x_1(t) + c_2 x_2(t)
$$

-- is delayed compared to the chest by \(d\) samples (non-integer!). We denote this as

$$
y(t) = x_2(t + \tau) = x_2(t + dT)
$$

where \(\tau = dT\) is the amount of time the chest signal is ahead.

Clearly, if we know the time difference, we can time shift (delay) the chest signal, and then use the l2-norm based optimal rejection/cancellation method of the first problem. The [cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation) can be used to find the time delay at which the signals correlate with each other the most.

This time, however, we cannot go ahead and just shift the samples due to non-integer delay. To do the shifting, we must use interpolation. In this exercise, we use [spline interpolation](https://en.wikipedia.org/wiki/Spline_interpolation) in which the values between the samples are approximated using piece-wise polynomials fitted to the neighbouring data points. If the polynomial to be fitted is of the first order, we have [linear interpolation](https://en.wikipedia.org/wiki/Linear_interpolation). However, to better represent band-limited sampled signals, we use [cubic spline interpolation](https://se.mathworks.com/help/matlab/ref/spline.html).

### Relation to Adaptive Filtering

Now, note that the solution of the problem corresponds to the signal processing filter structure shown in the Figure below.

![Filter Structure](https://lcms-files.mathworks.com/content/images/13f7b70d-a1f8-4380-902a-375ff6eb527a.png)

Basically, in the linear interpolation case, we have here a multitap (adaptive) filter where only two of the coefficients are non-zero, and those two contain the optimal scale \(c_2\) divided between them in the ratio of how close the non-integer part of the time shift is to those lags. For example, if the shift were 1.5 samples (\(d = 1.5\)), the 2nd and 3rd coefficients of the filter would both be \(\frac{c_2}{2}\) all the others being zeros. For the more generic spline interpolation, there will be more non-zero coefficients depending on the order of the spline.

### Task

Load data from 'problem3.mat' file, and use the variables 'abd_sig1' and 'mhb_ahead'. The former vector contains an artificial mixture of maternal and fetal ECGs summed together in the abdomen measurement. The latter vector contains only the maternal ECG as measured from the chest.

First, fix the time delay by shifting the maternal ECG in the 'mhb_ahead' vector accordingly, storing the value in the vector 'mhb'. Pad the missing samples with the linear extrapolation value.

Note that to handle the non-integer delay, we interpolate the output of the cross-correlation computed from sampled signals into a continuous curve using splines, and then find the non-integer lag giving the highest correlation from this continuous curve using function minimization. Also, consider limiting the range of cross-correlation lags you compute and use for the spline fitting to get a nice fit for the spline and to be able to find the correct local minimum (hint: plot the cross-correlation to see why).

### Note

Please consult the MATLAB help on [spline](https://se.mathworks.com/help/matlab/ref/spline.html) and [fnmin](https://se.mathworks.com/help/curvefit/fnmin.html).


In [ ]:
"""
% Load problem3.mat to have access to variables abd_sig1 and mhb_ahead
load('problem3.mat');


% The sampling rates are 1000 Hz
FS = 1000;

% Calculate sample timing vector in seconds starting from 0
t = (0:length(abd_sig1)-1) / FS;


% Estimate the time lag using cross correlation with the 'xcorr' function
% Fit a spline to the cross correlation using 'spline' function, and then find the delay with maximum correlation using 'fnmin'
% NOTE: to use minimization function for maximization, please invert the objective function!
[r, time_lag] = xcorr(abd_sig1, mhb_ahead);
P = spline(time_lag, -r);
[~, index] = fnmin(P);
d = index;


% Shift the chest ECG mhb_ahead back in time d samples
% Use linear interpolation with extrapolation with the function 'interp1'
mhb = interp1(t - d, mhb_ahead, t, 'linear', 'extrap');

org_coord = 1:length(mhb_ahead);
new_coord = org_coord - d;
signal_delay = mhb_ahead;
mhb = interp1(org_coord, signal_delay, new_coord, 'linear', 'extrap')';

% Estimate c2 from abd_sig1 and mhb
c2 = sum(abd_sig1 .* mhb) / sum(mhb.^2);


% Calculate the fetal ECG by cancelling out the scaled mother's ECG using projection based estimation coefficient
fetus = abd_sig1 - c2 .* mhb;
"""

## 3.4 Full adaptive filtering

### Background

We have already seen that adaptive filter constructions can handle signal amplitude differences and time delays. This time, let us suppose that, in addition, there is also some power line interference in the maternal ECG recorded from the chest.

We use the LMS-filter to adapt to the signal, and to cancel out the maternal ECG from the fetal ECG. Chapters 3.6.1 and 3.6.2 of the course book show the derivation of the filter and its learning rule.

**Adaptive LMS-filter**

A digital adaptive filter is basically constructed by two main components, including a digital FIR filter with adjustable coefficients and an adaptation algorithm which modifies the coefficients in order to optimize the filter. In MATLAB, the DSP System Toolbox includes the `dsp.LMSFilter` object that implements the classic LMS-filter among others.

Figure 1 below depicts a schematic diagram of the adaptive filter where there is a primary observed signal \(x(n)\), which is a sum of interference \(m(n)\) and the signal of interest \(v(n)\). The reference signal is \(r(n)\), which correlates with the interfering component \(m(n)\).

$$
\begin{align*}
x(n) & = m(n) + v(n) \\
r(n) & = m(n)
\end{align*}
$$

![Adaptive Filter](https://lcms-files.mathworks.com/content/images/4fcfeb87-d6e9-4381-aeae-c8cb0245d998.png)

The aim of an adaptive filter is to find y(n) as close as possible to m(n). So, $$v(n) \approx x(n) - y(n)$$. Since $$y(n) \approx m(n)$$, the output would approximately be the signal of interest v(n).

It can be proved that the output e(n) is the minimum mean square error (MMSE) estimate of the signal of interest $$v(n)$$, if the FIR filter coefficients are adjusted according to the famous update rule of the LMS filter:

$$
w(n+1) = w(n) + 2\mu e(n) r(n) \quad (1)
$$

where μ is the step size of the algorithm controlling how big changes are allowed at each update step. The value of μ should be selected by experimentation, but to guarantee stability the allowed range of μ is (for a FIR filter with m coefficients):

$$
0 < \mu < \frac{1}{\sigma^2 \sum_{n=1}^{m} r^2(n)} : \mu \in [0, \mu_{\text{max}}] \quad (2)
$$

It is often practical to select μ as a fraction (0 < c < 1) of $$\mu_{\text{max}}$$, i.e. $$\mu = c\mu_{\text{max}}$$.

In contrast to the course book, [the step size](https://se.mathworks.com/help/dsp/ref/dsp.lmsfilter.maxstep.html) of MATLAB [dsp.LMSFilter](https://se.mathworks.com/help/dsp/ref/dsp.lmsfilter-system-object.html) includes the factor 2 in (1). Taking this into account, the step size can be set as:

$$
\text{step} = 2\mu = 2c\mu_{\text{max}} = \frac{2c}{\left(\frac{1}{m}\right) \sum_{n=1}^{m} r^2(n)} (3)
$$

showing that the selected fraction, signal power, and filter length are the factors needed to determine the step size.

The [maxstep](https://se.mathworks.com/help/dsp/ref/dsp.lmsfilter.maxstep.html) function can be used to estimate the maximum step size, but for this exercise, we use \(\mu_{\text{max}} = 0.05\) for all \(m\). To get the step size for each parameter combination, multiply it by \(2c\) and divide by \(m\).

### Task

Your task is to find the best parameter combination for LMS filtering. To accomplish that, complete the given function template by writing your own code that:

1. Goes through all the possible combinations of filter lengths in m_list and learning rate fractions in c_list.
2. Does the classic LMS filtering using doLMSFiltering() [nested function](https://se.mathworks.com/help/matlab/matlab_prog/nested-functions.html) (to be completed by you) for each combination of the parameters.
3. Compares the LMS filter output with each combination to the known fetalECG by computing the mean squared error by evaluateResult() [nested function](https://se.mathworks.com/help/matlab/matlab_prog/nested-functions.html) (to be completed by you).
4. Selects the best parameter combination giving the lowest MSE.
5. Returns the best parameters, the filter coefficients of the best filter, and the lowest MSE.

Hence, you have to complete:

- The first [nested function](https://se.mathworks.com/help/matlab/matlab_prog/nested-functions.html) doLMSFiltering() to create the dsp.LMSFilter object based on the parameters given to that function and to then use it on the data given in the said parameters.
- The second [nested function](https://se.mathworks.com/help/matlab/matlab_prog/nested-functions.html) evaluateResult() that you will use to evaluate how closely each LMS filter output corresponds to the known fetal ECG.
- The main body of the findBestFilterParameters() function calling doLMSFiltering() and evaluateResult() functions suitably on all the parameter combinations to be tested.

The data for this problem is stored in the file 'problem4.mat' in the variables 'abd_sig1' and 'mhb_ahead_PI', and 'fhb'. The variable 'abd_sig1' is an artificial mixture of maternal and fetal ECGs summed together in the abdomen measurement. The variable 'mhb_ahead_PI' only contains the maternal ECG as measured from the chest, including time delay and power line interference. The variable 'fhb' contains the known true fetal ECG that usually cannot be measured directly.

### Note

Please take into account that the mathematical variable namings have some differences between this and the previous three problems!



In [ ]:
"""
function [best_m, best_c, best_w, best_mse] = findBestFilterParameters(chestECG, abdomenECG, fetalECG, m_list, c_list, mu_max)
% This function finds the best LMS filter parameter combination from the
% given lists using two inner functions.
% To be completed by you!
%
% INPUTS:
%   chestECG    ECG from the chest, maternal ECG only, reference input, Nx1
%   abdomenECG  ECG from the abdomen, fetal and maternal mixed, primary input, Nx1
%   fetalECG    ECG from the fetus alone, signal of interest, Nx1 (cannot be measured directly, but is given for evaluation here)
%   m_list      list of filter lengths/orders to test, Mx1
%   c_list      list of step size fractions to test, Cx1 (each >0 & <1)
%   mu_max      maximum step size, scalar
%
% OUTPUTS:
%   best_m      the best filter length (from the m_list), scalar
%   best_c      the best fraction of mu_max (from the m_list), scalar
%   best_w      the best filter coefficients, best_m x 1 vector
%   best_mse    the lowest mean squared error obtained with the best parameters, scalar

% When evaluating the results in evaluateResult(), skip this many samples from the beginning to avoid initial adaptation transient
INITIAL_REJECTION = 2000;
fetalECG = fetalECG(INITIAL_REJECTION + 1:end);

% Here you go through all the possible combinations of filter lengths in m_list and learning rate fractions in c_list selecting the best performing one

best_mse = inf;
for m_index = 1:length(m_list)
    for c_index = 1:length(c_list)

        m_val = m_list(m_index);
        c_val = c_list(c_index);
        step = (2*c_val*mu_max)/m_val;
        r_val = chestECG;
        x_val = abdomenECG;
        [y,e,w] = doLMSFiltering(m_val,step,r_val,x_val);
        v_val = x_val - y;
        mse = evaluateResult(v_val);

        % Updating [best_m, best_c, best_w, best_mse] based on mse
        if mse < best_mse
            best_m = m_val;
            best_c = c_val;
            best_w = w;
            best_mse = mse;
        end
    end
end


    function [y,e,w] = doLMSFiltering(m,step,r,x)
    % Does the actual LMS filtering.
    % To be completed by you!
    %
    % INPUTS:
    %   m       filter length
    %   step    LMS learning rule step size
    %   r       reference input (to be filtered)
    %   x       primary observed signal
    %
    % OUTPUTS:
    %   y       filtered signal r
    %   e       filter output, estimate of the signal of interest v

    % Create the dsp.LMSFilter object and use it to filter the input data
    lmsFilter = dsp.LMSFilter('Length', m, 'StepSize', step);

    % Perform LMS filtering
    [y, e, w] = lmsFilter(r, x);
    end

    function mse = evaluateResult(v)
    % Calculates the mean squared error between the filtered signal v and
    % the known fetal ECG.
    %
    % NOTE1:    Skip INITIAL_REJECTION number of samples in the beginning of both signals to not include initial adaptation transient
    %
    % NOTE2:    This nested function can access the desired output value in fetalECG directly!
    %
    % INPUTS:
    %   v       estimate of the signal of interest
    %
    % OUTPUTS:
    %   mse     mean squared error between v and fetalECG

    % You can call the 'immse' function for the signals without the initial rejection parts
    v_new = v(INITIAL_REJECTION + 1:end);
    mse = immse(v_new, fetalECG);
    end
end
"""

In [ ]:
"""
load('problem4.mat');

MU_MAX = 0.05;
FILTER_LENGHTS = [1 5 11 15 21 31 51 101]';
ADAPTATION_RATES = (0.1:0.1:1)';

[best_m, best_c, best_w, best_mse] = findBestFilterParameters(mhb_ahead_PI, abd_sig1, fhb, FILTER_LENGHTS, ADAPTATION_RATES, MU_MAX)
"""